In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

2021-08-11 05:44:34.774783: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
df2 = pd.read_csv("./NER_multilabel_data_v2.csv")

In [4]:
df2[50:100]

,Unnamed: 0,Sentence #,Word,POS,Tag,newTag
50,50,NaN,the,DT,O,O
51,51,NaN,Bombings,NNS,O,O
52,52,NaN,.,.,O,O
53,53,NaN,"""",``,O,O
54,54,Sentence: 3,They,PRP,O,O
55,55,NaN,marched,VBD,O,O
56,56,NaN,from,IN,O,O
57,57,NaN,the,DT,O,O
58,58,NaN,Houses,NNS,O,O
59,59,NaN,of,IN,O,O


In [ ]:
k = 0
df4 = pd.DataFrame(columns=['Unnamed: 0', 'Sentence #', 'Word', 'POS', 'Tag', 'newTag', 'tags_v4'])
for _, row in tqdm(df2.iterrows(), total=df2.shape[0]):
    words = tokenizer.tokenize(row.Word)
    for i, word in enumerate(words):
        if row.newTag != "O":
            if i == 0 and "I-" not in row.newTag:
                tags = row.newTag.split("|")

                tags = list(map(lambda x: f"B-{x}" if x[1] != "-" else x, tags))

                row["tags_v4"] = "|".join(tags)
            else:
                tags = row.newTag.split("|")

                tags = list(map(lambda x: f"I-{x}" if x[1] != "-" else x, tags))

                row["tags_v4"] = "|".join(tags)
        else:
            row["tags_v4"] = row.newTag
        if i == 0 and pd.isna(row["Sentence #"]):
            word = "Ġ"+word
        if i != 0:
            row["Sentence #"] = float("nan")
        row.Word = word
        #df4 = df4.append(row)
        df4.loc[k] = row
        k += 1

  3%|█                               | 36376/1048575 [04:03<2:35:36, 108.41it/s]

In [ ]:
#fix fullStop's problem
df4.Word = df4.Word.apply(lambda x: "." if x == "Ġ." else x)

In [ ]:
df4.to_csv("./NER_multilabel_data_v4.csv")

In [ ]:
from telegram_notifier import send_message as telegram_bot_sendtext
telegram_bot_sendtext("v4已儲存")

df3["Sentence #"] = (df3["Sentence #"].apply(lambda x: x if x else float("nan")))